### Pytorch introduction TLDR

A very simple pytorch CNN classifier.

#### Import needed packages

In [ ]:
from __future__ import print_function
import torch, torch.utils.data as data, torch.nn as nn
import torchvision,torchvision.transforms as transforms, torchvision.datasets as datasets

#### Define CNN

In [ ]:
class Lenet5(torch.nn.Module):
    def __init__(self):
        super(Lenet5, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, (5,5))
        self.conv2 = torch.nn.Conv2d(6, 10, (5,5))
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.fc1 = torch.nn.Linear(160, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc_class = torch.nn.Linear(84, 10)

    def forward(self, X):
        X=self.pool(torch.tanh(self.conv1(X)))
        X=self.pool(torch.tanh(self.conv2(X)))
        X=X.view([X.size()[0],160])
        X=torch.tanh(self.fc1(X))
        X=torch.tanh(self.fc2(X))
        return self.fc_class(X)

#### Load datasets

In [ ]:
trainloader = data.DataLoader(datasets.MNIST(
    root='./data', train=True, download=True, transform=transforms.ToTensor()), batch_size=256, shuffle=False)
testloader = data.DataLoader(datasets.MNIST(
    root='./data', train=False, download=True, transform=transforms.ToTensor()), batch_size=10000)

#### Train CNN

In [ ]:
net=Lenet5()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=.01, momentum=0.9)

for epoch in range(30):
    for batch_idx, (X,y_gt) in enumerate(trainloader):
        optimizer.zero_grad() # clear gradients
        y_out = net(X)
        loss = criterion(y_out, y_gt) # compute error
        loss.backward() # compute weiht updates with respect to error
        optimizer.step() # apply weight updates
    inputs, targets = iter(testloader).next()
    outputs = net(inputs)
    _, predicted = outputs.max(1)
    print ("Epoch {}:  Validation Accuracy {}%".format(epoch+1,predicted.eq(targets).sum().item()/10000.0))